### Import

In [805]:
import os
import re

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

### 1. Учим conv сеть для классификации - выбить auc выше 0.95

##### Data import

In [806]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [807]:
df = pd.read_excel('отзывы за лето.xls')

In [808]:
df.head(3)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14


In [809]:
df['class'] = -1
df.loc[df['Rating'] < 3, 'class'] = 0
df.loc[df['Rating'] > 3, 'class'] = 1

In [810]:
df = df.loc[df['class'] != -1]

In [811]:
df[10:15]

,Rating,Content,Date,class
10,5,Все ок!,2017-08-14,1
11,5,"Все нормально, кроме того что уведомление нель...",2017-08-14,1
12,2,"Не стартует без доступа к gps, sms, звонкам и ...",2017-08-14,0
13,5,"Очень удобно, работает замечательно, подвисани...",2017-08-14,1
14,5,Очень удобно,2017-08-14,1


In [812]:
df.drop(columns=['Rating', 'Date'], inplace=True)
df.rename(columns={'Content': 'text'}, inplace=True)

In [813]:
df[:3]

,text,class
0,It just works!,1
1,В целом удобноное приложение...из минусов хотя...,1
2,Отлично все,1


In [814]:
df_train, df_test = train_test_split(df, test_size=0.001, shuffle=True)
df_train, df_val = train_test_split(df_train, test_size=0.1, shuffle=True)

In [815]:
df_train.head(3)

,text,class
3285,Долго,1
17948,"Добавьте,пожалуйста, оформление частичной доср...",1
5629,Все отлично работает,1


In [816]:
df_test.head(3)

,text,class
15023,Класс,1
226,Хорошее приложение. Всё удобно и быстро,1
1860,Все ок,1


In [817]:
df_val.head(3)

,text,class
10291,"Просто и оперативно, спасибо!",1
6853,Молодцы,1
556,Norm,1


##### Preprocessing

In [818]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

C:\Users\Mihail\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [819]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [820]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mihail\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [821]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [822]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [823]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [824]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [825]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [826]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [827]:
x_train.shape

(17755, 40)

In [828]:
max_len

40

In [829]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
       107])

##### Объявляем модель

In [830]:
import numpy as np
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import activations

from tensorflow import keras as K

In [831]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [909]:
model = Sequential()
model.add(Embedding(input_dim=250, output_dim=328, input_length=max_len))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))


# model.add(Conv1D(64, 3))
# model.add(Activation(activations.tanh))



model.add(GlobalMaxPool1D())

model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [910]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.Adam(learning_rate=0.0045),
              metrics=['accuracy'])

##### Обучаем модель

In [913]:
epochs = 9

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=1000,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.01)

Epoch 1/3
18/18 [==============================] - 23s 1s/step - loss: 0.1920 - accuracy: 0.9193 - val_loss: 0.1741 - val_accuracy: 0.9157
Epoch 2/3


KeyboardInterrupt: 

##### roc_auc

In [914]:
results = model.predict(x_val, batch_size=batch_size, verbose=1)

results

roc_auc_score([val[1] for val in y_val], [val[1] for val in results])

4/4 [==============================] - 1s 174ms/step


0.9509003083819485

### 2. Берём предобученный word2vec и его эмбедингами инициализируем нейросеть, как влияет на качество?

##### Обучаем word2vec

In [977]:
model.layers[0].weights[0].shape

TensorShape([250, 328])

In [948]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [949]:
vect = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', lowercase=False)

In [950]:
train_ft = vect.fit_transform(df_train['text'])
valid_ft = vect.transform(df_val['text'])

In [955]:
from gensim.models import Word2Vec

In [956]:
modelW2V = Word2Vec(sentences=df_train['text'].apply(str.split), size=328, window=5, min_count=5, workers=8)

In [958]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['text'])
tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

In [960]:
rt = vect_idf.vocabulary_.items()

In [961]:
len(tfidf)

11523

In [962]:
from collections import defaultdict

In [963]:
max_idf = max(vect_idf.idf_)

word2weight = defaultdict(
    lambda: max_idf,
    [(w, vect_idf.idf_[i]) for w, i in vect_idf.vocabulary_.items()])

In [964]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(328)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

def get_vect_idf(txt):
    vector_w2v = np.zeros(128)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            iddf_ = tfidf.get(wrd, 1.)
            vector_w2v += modelW2V[wrd]*iddf_
            n_w2v += iddf_
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [965]:
from tqdm import tqdm_notebook

In [966]:
arr_vect = []
for txt in tqdm_notebook(df_train['text']):
    arr_vect.append(get_vect_mean(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['text']):
    arr_vect_valid.append(get_vect_mean(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

C:\Users\Mihail\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\Mihail\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
C:\Users\Mihail\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


C:\Users\Mihail\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [967]:
train_w2v.shape

(17755, 328)

In [968]:
train_w2v = train_w2v[:250]

In [969]:
train_w2v.shape

(250, 328)

##### Инициализируем модель обученными w2v эмбеддингами.

In [992]:
model.layers[0].set_weights([train_w2v])

In [993]:
results = model.predict_proba(x_val, batch_size=batch_size, verbose=1)

roc_auc_score([val[1] for val in y_val], [val[1] for val in results])

4/4 [==============================] - 1s 170ms/step


0.7696086807928912

##### Инициализируем слой эмбеддинга нейросети рандомными значениями.

In [982]:
model.layers[0].set_weights([np.random.rand(250, 328)])

In [987]:
for ind in range(10):
    model.layers[0].set_weights([np.random.rand(250, 328)])

    results = model.predict_proba(x_val, batch_size=batch_size, verbose=1)

    print(round(roc_auc_score([val[1] for val in y_val], [val[1] for val in results]), 5))

Instructions for updating:
Please use `model.predict()` instead.
4/4 [==============================] - 1s 184ms/step
0.47309
4/4 [==============================] - 1s 180ms/step
0.22963
4/4 [==============================] - 1s 237ms/step
0.21989
4/4 [==============================] - 1s 190ms/step
0.21703
4/4 [==============================] - 1s 182ms/step
0.32181
4/4 [==============================] - 1s 182ms/step
0.36409
4/4 [==============================] - 1s 239ms/step
0.29368
4/4 [==============================] - 1s 197ms/step
0.26463
4/4 [==============================] - 1s 187ms/step
0.21028
4/4 [==============================] - 1s 229ms/step
0.31957


##### Инициализируем нейросеть эмбеддингами w2v и обучим.

In [995]:
model = Sequential()
model.add(Embedding(input_dim=250, output_dim=328, input_length=max_len))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))


# model.add(Conv1D(64, 3))
# model.add(Activation(activations.tanh))



model.add(GlobalMaxPool1D())

model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [996]:
model.layers[0].set_weights([train_w2v])

In [997]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.Adam(learning_rate=0.0045),
              metrics=['accuracy'])

In [1000]:
epochs = 3

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=1000,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.01)

Epoch 1/3
18/18 [==============================] - 26s 1s/step - loss: 0.1951 - accuracy: 0.9181 - val_loss: 0.1842 - val_accuracy: 0.9157
Epoch 2/3
18/18 [==============================] - 29s 2s/step - loss: 0.1917 - accuracy: 0.9207 - val_loss: 0.1881 - val_accuracy: 0.8933
Epoch 3/3
18/18 [==============================] - 27s 2s/step - loss: 0.1922 - accuracy: 0.9201 - val_loss: 0.1904 - val_accuracy: 0.8764


In [1001]:
results = model.predict_proba(x_val, batch_size=batch_size, verbose=1)

roc_auc_score([val[1] for val in y_val], [val[1] for val in results])

4/4 [==============================] - 1s 199ms/step


0.9441961801967922

##### Инициализируем нейросеть эмбеддингами w2v, заморозим слой и обучим нейросеть.

In [1002]:
model = Sequential()
model.add(Embedding(input_dim=250, output_dim=328, input_length=max_len))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))
model.add(Conv1D(147, 3))
model.add(Activation(activations.relu))
# model.add(Dropout(0.025))


# model.add(Conv1D(64, 3))
# model.add(Activation(activations.tanh))



model.add(GlobalMaxPool1D())

model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [1003]:
model.layers[0].set_weights([train_w2v])
model.layers[0].trainable = False

In [1004]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.Adam(learning_rate=0.0045),
              metrics=['accuracy'])

In [1007]:
epochs = 3

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=1000,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.01)

Epoch 1/3
18/18 [==============================] - 27s 1s/step - loss: 0.3435 - accuracy: 0.8586 - val_loss: 0.3555 - val_accuracy: 0.8539
Epoch 2/3
18/18 [==============================] - 24s 1s/step - loss: 0.3423 - accuracy: 0.8608 - val_loss: 0.3616 - val_accuracy: 0.8539
Epoch 3/3
18/18 [==============================] - 22s 1s/step - loss: 0.3425 - accuracy: 0.8581 - val_loss: 0.3607 - val_accuracy: 0.8708


In [1008]:
results = model.predict_proba(x_val, batch_size=batch_size, verbose=1)

roc_auc_score([val[1] for val in y_val], [val[1] for val in results])

4/4 [==============================] - 1s 182ms/step


0.792171032761608

Разморозим слой, дообучим сеть.

In [1010]:
model.layers[0].trainable = True

In [1018]:
epochs = 10

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=1000,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.01)

Epoch 1/10
18/18 [==============================] - 21s 1s/step - loss: 0.3227 - accuracy: 0.8713 - val_loss: 0.3504 - val_accuracy: 0.8539
Epoch 2/10
18/18 [==============================] - 25s 1s/step - loss: 0.3186 - accuracy: 0.8712 - val_loss: 0.3339 - val_accuracy: 0.8652
Epoch 3/10
18/18 [==============================] - 25s 1s/step - loss: 0.3173 - accuracy: 0.8708 - val_loss: 0.3281 - val_accuracy: 0.8764
Epoch 4/10
18/18 [==============================] - 25s 1s/step - loss: 0.3179 - accuracy: 0.8734 - val_loss: 0.3395 - val_accuracy: 0.8708
Epoch 5/10
18/18 [==============================] - 24s 1s/step - loss: 0.3123 - accuracy: 0.8755 - val_loss: 0.3193 - val_accuracy: 0.8596
Epoch 6/10
18/18 [==============================] - 24s 1s/step - loss: 0.3093 - accuracy: 0.8754 - val_loss: 0.3215 - val_accuracy: 0.8596
Epoch 7/10
18/18 [==============================] - 24s 1s/step - loss: 0.3059 - accuracy: 0.8763 - val_loss: 0.3097 - val_accuracy: 0.8652
Epoch 8/10
18/18 [==

In [1019]:
results = model.predict_proba(x_val, batch_size=batch_size, verbose=1)

roc_auc_score([val[1] for val in y_val], [val[1] for val in results])

4/4 [==============================] - 1s 185ms/step


0.8601720342081418

##### Вывод по использованию w2v при инициализации слоя эмбеддинга.

КОММЕНТАРИИ:

Уже во второй домашней работе просят инициализировать эмбеддинг нейросети эмбеддингами, обученными через word2vec и во второй раз это ухудшает качество работы сети. С одной стороны ожидаемо, что ухудшает - все-таки в "сработавшуюся" нейросеть засовывают чужеродные веса. С другой стороны второй раз фигурирует этот вопрос, как будто бы ожидается, что качество должно вырасти.

Плюс у меня нет уверенности, что корректно инициализирую, так что, возможно, если все делать правильно, то качество действительно должно улучшиться.

По факту имеем: нейросеть инициализированная рандомными эмбеддингами учится хорошо и до приличных 0.95 по ROC AUC, инициализированная w2v векторами нейросеть сходу выдает 0.76, для сравнения рандомная инициализация эмбедингов выдает в районе 0,3 ROC AUC. Если попробовать заморозить слой эмбеддинга после этого и продолжить обучение, нейросеть не уходит далеко от 0,76 изначальных по ROC AUC, хотя как показал опыт, после размораживания слоя нейросеть начинает повышать качество при дальнейшем обучении, но делает это намного медленней, чем при рандомных эмбеддингах изначальных.

ВЫВОДЫ:

- Способ инициализации весов нейросети влияет на скорость и характер её дальнейшего обучения.
- Использование word2vec обученных эмбеддингов при использовании нейросетевых моделей не оправдано.